# Case Study 1 : Collecting Data from Twitter

Due Date: 2/8/2021, 

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** 

    Kate Sincaglia
    
    Mitchell Sirois
    
    Bryson Tang
    
    Jyalu Wu

# Problem 1 (20 points): Sampling Twitter Data with either Search or Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [120]:
import twitter
#---------------------------------------------
# A Function to Login Twitter API
def oauth_login():
    
    CONSUMER_KEY = 'oVek8jmhI4sz8SbKnT1e6UsIh'
    CONSUMER_SECRET = '3eGB6ip82K3XowIDEmz9JXIbRc2uFSlHvjPJ8Lmkf3HVIdlmhM'
    OAUTH_TOKEN = '571213367-H0r2WfySE7L4vgUJHv4YLW2vZGLa7TEPP7iWsTNu'
    OAUTH_TOKEN_SECRET = 'uKcYixkDRaCUvaP2oqH9a9fIaIB9nf4NKz32iqo2t7B7K'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------

import json

q = 'outage' #searching for power outages

count = 1000 #number of respones per page of search results

search_results = twitter_api.search.tweets(q=q, count=count)

statuses = search_results['statuses']

for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Store the collected tweets
collected_tweets = json.dumps(statuses, indent=4)



Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500


### Report some statistics about the tweets you collected 

* Our topic of interest was on power outages and we collected 5000 tweets

# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [77]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import time

status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]
    
      
from prettytable import PrettyTable
from collections import Counter

#creates table of top 30 words and their counts from our collected tweets
label, data = ('Word', words)
pt = PrettyTable(field_names=[label, 'Count']) 
c = Counter(data)
[ pt.add_row(kv) for kv in c.most_common()[:30] ]
pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment

print(pt)

+---------------+-------+
| Word          | Count |
+---------------+-------+
| outage        |   286 |
| a             |   202 |
| the           |   201 |
| in            |   171 |
| RT            |   160 |
| to            |   145 |
| is            |   144 |
| and           |   108 |
| an            |   103 |
| power         |   100 |
| for           |    96 |
| at            |    87 |
| of            |    83 |
| on            |    74 |
| BAUER         |    74 |
| OUTAGE        |    71 |
| @CityPowerJhb |    58 |
| been          |    57 |
| are           |    55 |
| Bauer         |    53 |
| I             |    51 |
| LA            |    51 |
| IN            |    50 |
| @JustRyCole:  |    45 |
| out           |    44 |
| you           |    44 |
| have          |    43 |
| We            |    43 |
| will          |    41 |
| Outage        |    41 |
+---------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [122]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


retweets = [
            # Store out a triple of these three values ...
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text']) 
            
            # ... for each status ...
            for status in statuses 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

# Display each item in a triple in a table

pt = PrettyTable(field_names=['Retweet Count', 'Screen Name','Text'])
[ pt.add_row(row) for row in sorted(retweets, reverse=True)[:5000] ]
pt.max_width['Text'] = 70
pt.align= 'l'

apt = pt #altered pretty table to remove duplicate tweets without altering original table

#cleans the first x rows of the table
def new_clean(the_table, x):
    for i in range(x):
        r = i + 1
        t = r + 1
        while the_table.get_string(start = i, end = r) == the_table.get_string(start = r, end = t):
                the_table.del_row(r)
new_clean(apt, 12)
print(apt.get_string(start = 0, end = 10))

+---------------+-------------+------------------------------------------------------------------------+
| Retweet Count | Screen Name | Text                                                                   |
+---------------+-------------+------------------------------------------------------------------------+
| 478           | DaKidGowie  | RT @DaKidGowie: To this day, no one can convince me that the power     |
|               |             | outage wasn't the NFL's attempt to kill the Ravens' momentum. 😒       |
| 328           | JFriks      | RT @JFriks: So what movie got stocked in while there was power outage? |
|               |             | https://t.co/N8nIEmIRWo                                                |
| 57            | JustRyCole  | RT @JustRyCole: BAUER OUTAGE IN LA                                     |
| 43            | coinbase    | RT @coinbase: On January 29th, https://t.co/0tiBsD1C1r had an outage   |
|               |             | and many users experienc

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary



* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [6]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


*------------------------

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [9]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Come up with a business question and describe how Twitter data and a NoSQL database such as MongoDB call help you anser it.


---------------------------------------------------------------------------
**Slides (for 10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 


